EDA of DB23 (`ncdd_admin_database_25provinces__2023.xlsx` from Lok on 20231212)

In [2]:
import pandas as pd
import numpy as np
from pathlib import Path

In [7]:
%load_ext autoreload
%autoreload 2

In [11]:
from utils.explore_DB23 import *

In [3]:
# paths
code_path = Path().absolute()
datafd_path = code_path.parent.parent / 'data'

## Initial EDA of DB23

### Read in DB23 and stack all sheets vertically into `db_df`

In [14]:
DB23_path = datafd_path / 'other' / 'ncdd_admin_database_25provinces__2023.xlsx'
DB23_df_dic = pd.read_excel(DB23_path, sheet_name=None, header=2)  
# all sheets, key=sheet name, skip first 2 rows, make row 3 header

In [15]:
DB23_df_dic.keys()

dict_keys(['01. Banteay Meanchey', '02. Battambang', '03. Kampong Cham', '04. Kampong Chhnang', '05. Kampong Speu', '06. Kampong Thom', '07. Kampot', '08. Kandal', '09. Koh Kong', '10. Kratie', '11. Mondul Kiri', '12. Phnom Penh', '13. Preah Vihear', '14. Prey Veng', '15. Pursat', '16. Ratanak Kiri', '17. Siemreap', '18. Preah Sihanouk', '19. Stung Treng', '20. Svay Rieng', '21. Takeo', '22. Oddar Meanchey', '23. Kep', '24. Pailin', '25. Tboung Khmum'])

In [16]:
sht_name_lst = list(DB23_df_dic.keys())
db_prov_df_lst = [DB23_df_dic[sht] for sht in sht_name_lst]

In [23]:
# prov1_df = db_prov_df_lst[0]
# prov1_columns = prov1_df.columns
# nrow = prov1_df.shape[0]
# for df in db_prov_df_lst[1:]:
#     assert (df.columns == prov1_columns).all()
#     print(df.shape)
#     nrow = nrow + df.shape[0]
# nrow

(958, 7)
(1066, 7)
(648, 7)
(1461, 7)
(855, 7)
(593, 7)
(1150, 7)
(155, 7)
(382, 7)
(118, 7)
(1072, 7)
(291, 7)
(1295, 7)
(582, 7)
(302, 7)
(1021, 7)
(145, 7)
(177, 7)
(778, 7)
(1231, 7)
(337, 7)
(25, 7)
(102, 7)
(946, 7)


16424

In [21]:
db_df = pd.concat(db_prov_df_lst)

,Type,Code,Name (Khmer),Name (Latin),Reference,Official Note,Note (by Checker)
0,ស្រុក,102,មង្គលបូរី,Mongkol Borei,ប្រកាសលេខ ៤៩៣ប្រ.ក,NaN,NaN
1,ឃុំ,10201,បន្ទាយនាង,Banteay Neang,ប្រកាសលេខ ៤៩៣ ប្រ.ក,NaN,NaN
2,ភូមិ,1020101,អូរធំ,Ou Thum,ប្រកាសលេខ ៤៩៣ ប្រ.ក,NaN,NaN
3,ភូមិ,1020102,ភ្នំ,Phnum,ប្រកាសលេខ ៤៩៣ ប្រ.ក,NaN,NaN
4,ភូមិ,1020103,បន្ទាយនាង,Banteay Neang,ប្រកាសលេខ ៤៩៣ ប្រ.ក,NaN,NaN
...,...,...,...,...,...,...,...
941,ភូមិ,25071411,ទន្លេបិទក្រោម,Tonle Bet Kraom,Royal Degree 1445 on 31 Dec 2013 Move to Tboun...,Royal Degree 1445 on 31 Dec 2013 Move to Tboun...,NaN
942,ភូមិ,25071412,ទន្លេបិទលើ,Tonle Bet Leu,Royal Degree 1445 on 31 Dec 2013 Move to Tboun...,Royal Degree 1445 on 31 Dec 2013 Move to Tboun...,NaN
943,ភូមិ,25071413,យាយស,Yeay Sar,Royal Degree 1445 on 31 Dec 2013 Move to Tboun...,Royal Degree 1445 on 31 Dec 2013 Move to Tboun...,NaN
944,ភូមិ,25071414,ទន្លេបិទ,Tonle Bet,Royal Degree 1445 on 31 Dec 2013 Move to Tboun...,Royal Degree 1445 on 31 Dec 2013 Move to Tboun...,NaN
